In [ ]:
from src.visualise import settings
from src.visualise.plot import plot_data
from src.data.paths import project_dir
from src.data.analysis import read_tiff_img, Circle, create_circular_mask
from src.data.detector import find_circle_hough_method, img_for_circle_detection

import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
import re

from dataclasses import dataclass, field
from pathlib import Path

from copy import deepcopy

# Data structures

In [ ]:
@dataclass(frozen=True)
class DetectorImage:
    image: np.ndarray
    path: Path

    @property
    def init_circle(self) -> Circle:
        return Circle(x=self.image.shape[0]//2, y=self.image.shape[1]//2, r=100)

@dataclass(frozen=True)
class DetectorData:
    raw: DetectorImage
    lv: DetectorImage
    det_no: int
    circle: Circle = field(default=Circle())

@dataclass(frozen=True)
class DetectorDataCollection:
    path: Path
    data: dict[int, DetectorData] = field(default_factory=dict)

    def __post_init__(self):
        if not self.data:
            self._load_data()

    def _load_data(self):
        for file_path in sorted(self.path.iterdir()):
            if file_path.name.endswith('lv'):
                # get detector data
                det_id = re.findall(r'\d+', file_path.name)[0]
                det_no = int(det_id)
                # live view images
                lv_path = next(file_path.glob('**/*tif'))
                lv_data = read_tiff_img(lv_path, border_px=0)
                lv_image = DetectorImage(image=lv_data, path=lv_path)
                # raw data images
                try:
                    raw_path = next((self.path / det_id).glob('**/*tif'))
                    raw_data = read_tiff_img(raw_path, border_px=0)
                    raw_image = DetectorImage(image=raw_data, path=raw_path)
                    det_data = DetectorData(raw=raw_image, lv=lv_image, det_no=det_no)
                    self.data[det_no] = det_data
                    print(f"{det_no} ", end='')
                except StopIteration:
                    print(f"missing_{det_no} ", end='')

In [ ]:
raw_path = project_dir / 'data' / 'raw' / '2024-02-20'
proton_raw_data = DetectorDataCollection(path=raw_path)

In [ ]:
co60_path = project_dir / 'data' / 'raw' / 'Co60'
co60_raw_data = DetectorDataCollection(path=co60_path)

# Proton raw data

In [ ]:
fig, ax = plot_data(proton_raw_data.data[1].lv.image, path='', circle_px=proton_raw_data.data[1].lv.init_circle, figsize=(10,12))

In [ ]:
fig, ax = plot_data(proton_raw_data.data[1].raw.image, path='', circle_px=proton_raw_data.data[1].raw.init_circle, figsize=(10,12))

# Co60 raw data

In [ ]:
fig, ax = plot_data(co60_raw_data.data[1].lv.image, path='', circle_px=co60_raw_data.data[1].lv.init_circle, figsize=(10,12))

In [ ]:
fig, ax = plot_data(co60_raw_data.data[1].raw.image, path='', circle_px=co60_raw_data.data[1].raw.init_circle, figsize=(10,12))

## Background

In [ ]:
background_path = next(raw_path.parent.glob('**/*background*/**/**/*tif'))
background_data = DetectorImage(path=background_path, image=read_tiff_img(background_path, border_px=0))

In [ ]:
fig, ax = plot_data(background_data.image, path='', circle_px=background_data.init_circle, figsize=(10,12))

# Background subtraction

In [ ]:
proton_bg_sub_data = DetectorDataCollection(path=proton_raw_data.path, data=deepcopy(proton_raw_data.data))
for data in proton_bg_sub_data.data.values():

    # out data are save as uint16, dataclasses are frozen
    # its not straightforward to use np.crop(0) or cast to int64
    # therefore we shift data up, perform background subtraction, crop negative values and shift back
    np.add(data.raw.image, background_data.image.max(), out=data.raw.image)    
    np.subtract(data.raw.image, background_data.image, out=data.raw.image)
    np.clip(data.raw.image, a_min=background_data.image.max(), a_max=None, out=data.raw.image)
    np.subtract(data.raw.image, background_data.image.max(), out=data.raw.image)


In [ ]:
co60_bg_sub_data = DetectorDataCollection(path=co60_raw_data.path, data=deepcopy(co60_raw_data.data))
for data in co60_bg_sub_data.data.values():

    # out data are save as uint16, dataclasses are frozen
    # its not straightforward to use np.crop(0) or cast to int64
    # therefore we shift data up, perform background subtraction, crop negative values and shift back
    np.add(data.raw.image, background_data.image.max(), out=data.raw.image)    
    np.subtract(data.raw.image, background_data.image, out=data.raw.image)
    np.clip(data.raw.image, a_min=background_data.image.max(), a_max=None, out=data.raw.image)
    np.subtract(data.raw.image, background_data.image.max(), out=data.raw.image)

In [ ]:
fig, ax = plot_data(proton_raw_data.data[1].raw.image, path='', circle_px=proton_raw_data.data[1].raw.init_circle, figsize=(10,12))

In [ ]:
fig, ax = plot_data(proton_bg_sub_data.data[1].raw.image, path='', circle_px=proton_bg_sub_data.data[1].raw.init_circle, figsize=(10,12))

# Detector discovery

In [ ]:
find_circle_hough_method(lv_for_detect)

In [ ]:
fig, ax = plot_data(proton_raw_data.data[1].lv.image, path='', circle_px=find_circle_hough_method(lv_for_detect), figsize=(10,12))

In [ ]:
fig, ax = plot_data(proton_raw_data.data[1].raw.image, path='', circle_px=find_circle_hough_method(lv_for_detect), figsize=(10,12))

In [ ]:
det_data_dict = {}
for data in proton_raw_data.data.values():
    lv_for_detect = img_for_circle_detection(data.lv.image)
    circle = find_circle_hough_method(lv_for_detect)
    det_data = DetectorData(raw=data.raw, lv=data.lv, det_no=data.det_no, circle=circle)
    det_data_dict[data.det_no] = det_data
    print(f"{data.det_no} ", end='')
proton_det_data = DetectorDataCollection(path=proton_raw_data.path, data=det_data_dict)

In [ ]:
co60_data_bg_removed = (co60_data.astype(np.int64)-background_data.astype(np.int64)).clip(0,None)
mask_for_circle = create_circular_mask(img=co60_data_bg_removed, circle_px=Circle(c.x, c.y, 80))
co60_data_bg_removed_mean = np.mean(co60_data_bg_removed[mask_for_circle], where=co60_data_bg_removed[mask_for_circle]>0)
co60_data_bg_removed_std = np.std(co60_data_bg_removed[mask_for_circle], where=co60_data_bg_removed[mask_for_circle]>0)
co60_data_bg_removed_mean, co60_data_bg_removed_std / co60_data_bg_removed_mean

In [ ]:
dose_Co60 = 5
co60_data_bg_removed_mean / dose_Co60

In [ ]:
plot_data((proton_data.astype(np.int64)-background_data.astype(np.int64)).clip(0,None), path='', circle_px=Circle(c.x, c.y, 80))

In [ ]:
dose_proton_Gy = 5

proton_data_bg_removed = (proton_data.astype(np.int64)-background_data.astype(np.int64)).clip(0,None)
mask_for_circle = create_circular_mask(img=proton_data_bg_removed, circle_px=Circle(c.x, c.y, 80))
proton_data_bg_removed_mean = np.mean(proton_data_bg_removed[mask_for_circle], where=proton_data_bg_removed[mask_for_circle]>0)
proton_data_bg_removed_std = np.std(proton_data_bg_removed[mask_for_circle], where=proton_data_bg_removed[mask_for_circle]>0)
proton_data_bg_removed_mean

proton_data_bg_removed_mean / dose_proton_Gy, proton_data_bg_removed_std / proton_data_bg_removed_mean

# Efficiency

In [ ]:
co60_signal_per_Gy = (co60_data.astype(np.int64)-background_data.astype(np.int64)).clip(1,None) / dose_Co60
proton_signal_per_Gy = (proton_data.astype(np.int64)-background_data.astype(np.int64)).clip(1,None) / dose_proton_Gy
plot_data((proton_signal_per_Gy / co60_signal_per_Gy).clip(0.001,2), path='', circle_px=Circle(c.x, c.y, 80))

In [ ]:
(proton_data_bg_removed_mean / dose_proton_Gy) / (co60_data_bg_removed_mean / dose_Co60)

In [ ]:
plot_data(ndi.median_filter((proton_signal_per_Gy / co60_signal_per_Gy).clip(0.001,2),size=20), path='', circle_px=Circle(c.x, c.y, 80))